In [1]:
%load_ext autoreload
%autoreload 2

from Retrieval import format_context_for_llm
import pandas as pd

c:\Users\Gökhan ergül\Desktop\project\RAG\boeing_with_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Gökhan ergül\Desktop\project\RAG\boeing_with_rag\Retrieval.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [2]:
test_df = pd.read_csv('boeing_737_test_set.csv')
test_df

,Question,Labelled Answer,Page reference
0,I'm calculating our takeoff weight for a dry r...,"Based on the data for a dry runway at 2,000 fe...",83
1,"We're doing a Flaps 15 takeoff. Remind me, wha...","For a Flaps 15 takeoff, the first flap selecti...",41
2,We're planning a Flaps 40 landing on a wet run...,"For a Flaps 40 landing on a wet runway at 1,00...",99
3,Reviewing the standard takeoff profile: After ...,"After getting a positive rate of climb, the fi...","39,51"
4,Looking at the panel scan responsibilities for...,"Based on the ""Panel Scan Diagram"" for when the...",6
5,"For a standard visual pattern, what three acti...","According to the ""Visual Traffic Pattern"" diag...",56
6,If the PF is making entries into the CDU durin...,"Prior to execution, the Pilot Flying must veri...",5
7,"I see an amber ""STAIRS OPER"" light illuminated...",That amber STAIRS OPER (Operating) light indic...,126
8,We've just completed the engine start. What is...,"During the After Start Procedure, the ISOLATIO...",35
9,"During the Descent and Approach procedure, wha...",Here are the autobrake actions for those two p...,"43, 47"


In [3]:
test_df.iloc[8]['Question']

"We've just completed the engine start. What is the correct configuration for the ISOLATION VALVE switch during the After Start Procedure?"

In [4]:
#page 5: there is no answer,
#  35:there is no answer, 
# 43 only 47 not this 
from Retrieval import retrieve_similar_documents
query = test_df.iloc[9]['Question']
response = retrieve_similar_documents(query, k=5)
print(response)


--- Source 1 (Page 47) ---
**[Context Text]**
Landing Roll Procedure


**[Visual Analysis]**
| PILOT FLYING | PILOT NOT FLYING |
|---|---|
| Ensure thrust levers at idle. | |
| Disengage autopilot and control airplane manually. | Verify autothrottle is disengaged. |
| Verify autothrottle disengages automatically. | |
| Verify SPEED BRAKE lever (ground spoilers) - UP. | Verify SPEED BRAKE lever UP.<br>Call out “SPEED BRAKES UP.”<br>If SPEED BRAKE lever not UP,<br>call “SPEED BRAKES NOT UP.” |
| Verify proper autobrake operation. | |
| Without delay, raise reverse thrust levers to the interlocks, hold light pressure until release, and then apply reverse thrust as required. | Monitor engine instruments and announce any engine limit being approached, exceeded or any other abnormalities. |
| At 60 knots, reduce reverse thrust to be at IDLE reverse when reaching taxi speed. | Call “60 KNOTS” |
| Approaching taxi speed, slowly move the reverse thrust levers to the full down position. | Verif

In [5]:
import time 
for index, row in test_df.iterrows():
    question = row['Question']
    answer = row['Labelled Answer']
    Page_reference = row['Page reference']
    
    response = format_context_for_llm(question, k=8)
    
    print(f"Question: {question}")
    print(f"Expected Answer: {answer} expected on Page: {Page_reference}")
    print(f"Model Response: {response}")
    time.sleep(2)

    print("-" * 50)

Question: I'm calculating our takeoff weight for a dry runway. We're at 2,000 feet pressure altitude, and the OAT is 50°C. What's the climb limit weight ?
Expected Answer: Based on the data for a dry runway at 2,000 feet pressure altitude with an OAT of 50°C, the Climb Limit Weight is 52,200 kg. expected on Page: 83
Model Response: For a dry runway at 2,000 feet pressure altitude and an Outside Air Temperature (OAT) of 50°C, the climb limit weight is 51.3 (1000 KG). [Source: Page 83]
--------------------------------------------------
Question: We're doing a Flaps 15 takeoff. Remind me, what is the first flap selection we make during retraction, and at what speed?
Expected Answer: For a Flaps 15 takeoff, the first flap selection during retraction is Flaps 5, which you select at V2 + 15 knots. expected on Page: 41
Model Response: For a Flaps 15 takeoff, the first flap selection during retraction is Flaps 5, which is made at V2 + 15 knots [Source: Page 41].
-------------------------------

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 25.055278054s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5


Question: If the PF is making entries into the CDU during flight, what must the PF do prior to execution?
Expected Answer: Prior to execution, the Pilot Flying must verify the CDU entries made by the Pilot Not Flying. expected on Page: 5
Model Response: In flight, CDU entries are normally accomplished by the pilot not flying and verified by the pilot flying prior to execution. [Source: Page 5]
--------------------------------------------------
Question: I see an amber "STAIRS OPER" light illuminated on the forward attendant panel; what does that light indicate?
Expected Answer: That amber STAIRS OPER (Operating) light indicates that the airstair is in transit (i.e., currently extending or retracting). expected on Page: 126
Model Response: The amber "STAIRS Operating (OPER) Light" illuminates when the airstair is in transit (either extending or retracting) [Source: Page 126].
--------------------------------------------------
Question: We've just completed the engine start. What is the 

| ID | Question Topic      | Model Page | Expected Page | Page Score | Content Score | Notes                                                         |
|----|---------------------|------------|----------------|------------|---------------|--------------------------------------------------------------|
| 1  | Takeoff Weight      | 83         | 83             | 1.0        | 0.5           | Page found perfectly. Value read incorrect (51.3 vs 52.2).  |
| 2  | Flaps 15 Takeoff    | 41         | 41             | 1.0        | 1.0           | Perfect.                                                     |
| 3  | Flaps 40 Landing    | 99         | 99             | 1.0        | 1.0           | Perfect.                                                     |
| 4  | Positive Rate       | 51         | 39, 51         | 1.0        | 1.0           | Perfect citation of one of the valid pages.                 |
| 5  | Panel Scan          | 6          | 6              | 1.0        | 0.0           | Page found perfectly. Role swapped (FO vs Capt).            |
| 6  | Visual Pattern      | 56         | 56             | 1.0        | 1.0           | Perfect.                                                     |
| 7  | PF CDU Entries      | 5          | 5              | 1.0        | 1.0           | Perfect.                                                     |
| 8  | Stairs Light        | 126        | 126            | 1.0        | 1.0           | Perfect.                                                     |
| 9  | Isolation Valve     | -          | 35             | 0.0        | 0.0           | Retrieval Miss.                                              |
| 10 | Autobrake           | 47         | 43, 47         | 0.5        | 0.5           | Half point given. Found Landing page (47), missed (43).     |
